In [1]:
#Importing python libraries downloaded from the internet and those saved locally
import serial 
from serial.tools import list_ports
import yaml
import matplotlib
matplotlib.use('TkAgg') #This is a specific back end of matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2 as cv2
import time
import pandas as pd
import glob
import json
import datetime
from datetime import date
import utils.Plate_positions as pp #Local library
import ipywidgets as widgets
import threading

In [9]:
def getCameraIndices():
    """Returns valid camera indices for use with OpenCV"""
    index = 0
    arr = []
    i = 4
    while i > 0:
        try:
            cap = cv2.VideoCapture(index)
            if cap.read()[0]:
                arr.append(index)
                cap.release()
        except:
            print('exception')
        index += 1
        i -= 1
    return arr


In [2]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c == 27: #27 is the code for the escape key. So press escape to close the window
        break

cap.release()
cv2.destroyAllWindows()